In [ ]:
from collections import defaultdict
import numpy as np
from pychunkedgraph.graph import ChunkedGraph
from pychunkedgraph.graph.utils.inspect import dfs_print_node
from pychunkedgraph.graph import cache

from pychunkedgraph.graph.edits import add_edges, remove_edges
from pychunkedgraph.graph.utils.context_managers import TimeIt
from pychunkedgraph.graph.operation import MulticutOperation
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [ ]:
data = {
    "sources": [["88399772800622120", 729368.5, 847633.8125, 599680]],
    "sinks": [["88329404056440986", 728994.5, 847396.625, 599680]],
}
data = {
    "sources": [["94803466563722624",916778,851942.625,783023.25]], # 354
    "sinks": [["94803535283198313",916697.1875,852031.75,783023.25]], # 832 = 1186
}
data = {
    "sources": [["87979484480978348", 717418.8125, 905186.875, 602200]],
    "sinks": [["87979553200440097", 717687.6875, 905449.25, 602200]],
}
data_dict = {}
for k in ["sources", "sinks"]:
    data_dict[k] = defaultdict(list)
    for node in data[k]:
        node_id = node[0]
        x, y, z = node[1:]
        coordinate = np.array([x, y, z]) / cg.meta._ws_cv.resolution
        atomic_id = cg.get_atomic_id_from_coord(
            coordinate[0],
            coordinate[1],
            coordinate[2],
            parent_id=np.uint64(node_id),
        )

        if atomic_id is None:
            raise ValueError("aha")
        data_dict[k]["id"].append(atomic_id)
        data_dict[k]["coord"].append(coordinate)
data_dict        

In [ ]:
root = cg.get_root(data_dict["sources"]["id"][0])
print(root, cg.get_segment_id(root))
dfs_print_node(cg, root, limit=2, sep="    ", count_only=True)
cg.get_cross_chunk_edges_layer(
    [
        [data_dict["sources"]["id"][0], data_dict["sinks"]["id"][0]],
    ]
)

In [ ]:
with TimeIt("remove edges"):
    result = cg.remove_edges(
        user_id="hi",
        atomic_edges=None,
        source_coords=data_dict["sources"]["coord"],
        sink_coords=data_dict["sinks"]["coord"],
        source_ids=data_dict["sources"]["id"],
        sink_ids=data_dict["sinks"]["id"],        
    )

In [ ]:
print(result)
roots = result.new_root_ids
dfs_print_node(cg, roots[0], limit=2, sep="    ", count_only=True)
dfs_print_node(cg, roots[1], limit=2, sep="    ", count_only=True)